In [1]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/train-en-ms.tar.gz
# !tar -zxf train-en-ms.tar.gz

In [2]:
with open('train-en/left.txt') as fopen:
    left = fopen.read().split('\n')
    
with open('train-en/right.txt') as fopen:
    right = fopen.read().split('\n')

In [3]:
len(left), len(right)

(3807616, 3807616)

In [4]:
left = left[:500000]
right = right[:500000]

In [5]:
from tqdm import tqdm
import re

def cleaning(string):
    string = string.replace('\n', ' ').replace('\t', ' ')
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

In [6]:
import tensorflow as tf
import tensorflow_datasets as tfds
from t5.data import preprocessors as prep
import functools
import t5
import gin
import sentencepiece as spm
from glob import glob
import os

gin.parse_config_file('pretrained_models_base_operative_config.gin')
vocab = 'sp10m.cased.ms-en.model'
sp = spm.SentencePieceProcessor()
sp.Load(vocab)

True

In [7]:
with tf.io.gfile.GFile('en-ms.tsv', "w") as outfile:
    for i in range(len(left)):
        l = cleaning(left[i])
        r = cleaning(right[i])
        outfile.write("%s\t%s\n" % (l, r))

In [8]:
def en_ms_dataset(split, shuffle_files = False):
    del shuffle_files
    ds = tf.data.TextLineDataset(
        [
            'en-ms.tsv'
        ]
    )

    ds = ds.map(
        functools.partial(
            tf.io.decode_csv,
            record_defaults = ['', ''],
            field_delim = '\t',
            use_quote_delim = False,
        ),
        num_parallel_calls = tf.data.experimental.AUTOTUNE,
    )
    ds = ds.map(lambda *ex: dict(zip(['question', 'answer'], ex)))
    return ds

def en_ms_preprocessor(ds):
    def to_inputs_and_targets(ex):
        return {
            'inputs': tf.strings.join(['terjemah Inggeris ke Melayu: ', ex['question']]),
            'targets': ex['answer'],
        }

    return ds.map(
        to_inputs_and_targets,
        num_parallel_calls = tf.data.experimental.AUTOTUNE,
    )

In [9]:
t5.data.TaskRegistry.remove('en_ms_dataset')
t5.data.TaskRegistry.add(
    'en_ms_dataset',
    dataset_fn = en_ms_dataset,
    splits = ['train'],
    text_preprocessor = [en_ms_preprocessor],
    sentencepiece_model_path = vocab,
    metric_fns = [t5.evaluation.metrics.accuracy],
)


In [10]:
nq_task = t5.data.TaskRegistry.get("en_ms_dataset")
ds = nq_task.get_dataset(split='en-ms.tsv', sequence_length={"inputs": 1024, "targets": 1024})
r = tfds.as_numpy(ds)

In [11]:
next(r)

{'inputs_plaintext': b'terjemah Inggeris ke Melayu: However, the BMKG Majene reminded the people of Sulbar to remain alert.',
 'inputs': array([   13, 26087,  2040,    55,  1550,    31,  1027,    14,    15,
           13, 11615,   471,   541,    13, 14077,  8378, 16339,    15,
          121,    18, 13025,  1740,    19,  2653,  9392,     3,     1]),
 'targets_plaintext': b'Namun, BMKG Majene mengingatkan masyarakat di Sulbar untuk tetap waspada.',
 'targets': array([  918,    14,    13, 11615,   471,   541,    13, 14077,  8378,
         5252,   845,    24, 13025,  1740,    25,   800,    39,  6700,
            3,     1])}

In [12]:
!rm -rf train-en